# Exploratory Data Analysis

### TODO IMAGES MOVED TO MISC FIX RELATIVE PATHS THAT DONT WORK 

This notebook explores the dataset given for the 3D Breast Reconstruction Master Thesis Project

The code can be found at https://github.com/8toz/3d_breast_reconstruction/blob/main/EDA.ipynb

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.patches as mpatches

from scipy import ndimage
from tifffile import TiffFile
from shapely.affinity import scale, translate
from tqdm.notebook import tqdm_notebook


## Fine tune opencv to work with large images
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2

In [ ]:
GEOJSON_PATH = './data/GeoJson/H21-066.4_HE332_033_Scan1.qptiff - resolution #1.geojson'
GEOJSON_PATH_2 = './data/GeoJson/H21-066.4_HE332_225_Scan1#1.geojson'
IMAGE_PATH = './data/66-4/H21-066.4_HE332_033/processed_4/H21-066.4_HE332_033_Scan1.tif'
LABEL_DICT = {'epithelium':1, 'blood_vessels':2, 'stroma':3, 'adipocytes':4}
COLOR_DICT = {'epithelium':np.array([255, 0, 0]), 
              'blood_vessels':np.array([0, 0, 255]), 
              'stroma':np.array([0, 255, 0]) , 
              'adipocytes':np.array([255, 255, 0])}

In [ ]:
def find_files_with_extension(directory, extension):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_paths.append(os.path.join(root, file))
    return file_paths

def get_max_size(path):
    with TiffFile(path) as tif:
        height, width = -1, -1
        for i, page in enumerate(tif.pages):
            pages = str(tif.pages[i])
            img_height, img_width = get_resolution(pages)
            height = img_height if height < img_height else height
            width = img_width if width < img_width else width
    
    return height, width

def get_resolution(input_str):
    input_str = str(input_str) # Make sure is str format
    size = input_str.split(" ")[4]
    height, width = size.split("x")[0], size.split("x")[1]
    return int(height), int(width)

def get_histograms(ROOT):
    hist_b = np.zeros((256,1))
    hist_g = np.zeros((256,1))
    hist_r = np.zeros((256,1))

    hist_gray = np.zeros((256,1))

    images_list = os.listdir(ROOT)
    n_images = len(images_list)

    for img_path in os.listdir(ROOT):
        #RGB
        image = cv2.imread(os.path.join(ROOT, img_path))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        hist_b += cv2.calcHist([image_rgb], [0], None, [256], [0, 256])
        hist_g += cv2.calcHist([image_rgb], [1], None, [256], [0, 256])
        hist_r += cv2.calcHist([image_rgb], [2], None, [256], [0, 256])
        #BGR
        image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        hist_gray += cv2.calcHist([image_gray], [0], None, [256], [0, 256])

    combined_hist_b = hist_b/n_images
    combined_hist_g = hist_g/n_images 
    combined_hist_r = hist_r/n_images

    combined_hist_gray = hist_gray/n_images

    # Normalize the histograms
    hist_b_normalized = combined_hist_b / np.sum(combined_hist_b)
    hist_g_normalized = combined_hist_g / np.sum(combined_hist_g)
    hist_r_normalized = combined_hist_r / np.sum(combined_hist_r)
    hist_bw_normalized =  combined_hist_gray/np.sum(combined_hist_gray)

    return hist_b_normalized, hist_g_normalized, hist_r_normalized, hist_bw_normalized

def scale_geometry(geom, scaling_factor):
    return scale(geom, xfact=scaling_factor, yfact=scaling_factor, zfact=1, origin=(0, 0))

def load_labels(path, downscaling_factor=None):
    to_drop = ["id", "objectType", "classification", "object_type", "isLocked"]
    labels_df = gpd.read_file(path)
    names = list(labels_df["classification"][0].keys())
    labels_df["label"] = labels_df["classification"].apply(lambda x: x[names[0]])
    #labels_df["color"] = labels_df["classification"].apply(lambda x: rgb_to_hex(x[encoded_names[1]]))
    labels_df = labels_df.drop(columns=to_drop, errors="ignore")
    # Scale the coordinates
    if downscaling_factor is not None:
        labels_df['geometry'] = labels_df['geometry'].apply(lambda row: scale_geometry(row, (1/downscaling_factor)))

    # Encode and clean labels based in dict
    labels_df["label"] = labels_df["label"].str.lower()
    labels_df["label"] = labels_df["label"].str.replace(" ", "_")
    # Fix because labels are not consistent in the data
    labels_df['label'] = labels_df['label'].apply(lambda x: 'blood_vessels' if 'blood_vessel' in x else x)
    labels_df['label'] = labels_df['label'].apply(lambda x: 'adipocytes' if 'fat' in x else x)

    labels_df["encoded_label"] = labels_df["label"].apply(lambda x: LABEL_DICT[x])

    return labels_df

def load_image(img_path):
    image = cv2.imread(img_path)
    return image

def get_tif_paths(path, resolution):
    path_list = []
    for paths in os.walk(path, topdown=False):
        root = paths[0]
        if "processed_"+str(resolution) in root:
            for file in paths[-1]:
                path_list.append(os.path.join(root, file))
    return path_list
    
def get_labelled_files(labels_path, file_list):
    # Returns a tuple with image and labels associated with it
    file_label_list = []
    if len(file_list) == 0:
        raise("Create preprocessed files for this resolution")
    
    labelled_files = os.listdir(labels_path)
    for file in file_list:
        file_name = os.path.splitext(os.path.basename(file))[0]
        for labelled_file in labelled_files:
            if file_name in labelled_file:
                file_label_list.append((file, os.path.join(labels_path, labelled_file)))

    return file_label_list

def get_label_index(path):
    if "epithelium" in path:
        return LABEL_DICT["epithelium"]
    elif "blood_vessels" in path:
        return LABEL_DICT["blood_vessels"]
    elif "stroma" in path:
        return LABEL_DICT["stroma"]
    else:
        return LABEL_DICT["adipocytes"]

The EDA will have 2 sections:
- Exploring the image features
- Exploring the labels

## Image Features

Analysing image size

In [ ]:
height_list, width_list = [], []
for file in find_files_with_extension("./data/66-4", ".qptiff"):
    height, width = get_max_size(file)
    height_list.append(height)
    width_list.append(width)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))

sns.histplot(height_list, ax=ax1, color='blue')
sns.histplot(width_list, ax=ax2, color='orange')

mean_height = sum(height_list) / len(height_list)
mean_width = sum(width_list) / len(width_list)
ax1.axvline(mean_height, color='red', linestyle='--', label=f'Mean Height: {mean_height:.0f} pixels.')
ax2.axvline(mean_width, color='red', linestyle='--', label=f'Mean Width: {mean_width:.0f} pixels.')
ax1.set_title('Height Distribution')
ax1.set_xlabel('Height')
ax1.set_ylabel('Frequency')
ax2.set_title('Width Distribution')
ax2.set_xlabel('Width')
ax2.set_ylabel('Frequency')
ax1.legend()
ax2.legend()
plt.tight_layout()
plt.show()

We can apreciate there is an image whose size is significantly different for the rest.

In [ ]:
image = cv2.imread("./data/66-4/processed_32/H21-066.4_HE332_237_Scan1.tif")
cropped_image = image[2500:5000,500:2380]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 3))
ax1.set_title("Outlier")
ax1.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
ax2.set_title("Fixed outlier")
ax2.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
ax1.axis("off")
ax2.axis("off")
plt.tight_layout()
plt.show()

Now lets explore the histograms. We can either work on RGB format which would generate 3 histograms (one per channel) and B&W

In [ ]:
ROOT = "./data/66-4/processed_32"

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 4))

ROOT = "./data/66-4/processed_32"

hist_b_normalized, hist_g_normalized, hist_r_normalized, hist_bw_normalized = get_histograms(ROOT)

# Plot the normalized histograms
ax1.set_xlim(0, 256)
ax1.plot(hist_b_normalized, color='blue')
ax1.plot(hist_g_normalized, color='green')
ax1.plot(hist_r_normalized, color='red')
ax1.set_xlabel('Pixel Value')
ax1.set_ylabel('Normalized Frequency')
ax1.set_title('Normalized Histogram of RGB Image')
ax1.legend(['Blue Channel', 'Green Channel', 'Red Channel'])

# Plot the normalized histogram for the B&W image
ax2.set_xlim(0, 256)
ax2.plot(hist_bw_normalized, color='black')
ax2.set_xlabel('Pixel Value')
ax2.set_ylabel('Normalized Frequency')
ax2.set_title('Normalized Histogram of B&W Image')


# Plot the normalized histograms
ax3.set_xlim(200, 256)
ax3.plot(hist_b_normalized, color='blue')
ax3.plot(hist_g_normalized, color='green')
ax3.plot(hist_r_normalized, color='red')
ax3.set_xlabel('Pixel Value')
ax3.set_ylabel('Normalized Frequency')
ax3.set_title('Zoomed Histogram of RGB Image')
ax3.legend(['Blue Channel', 'Green Channel', 'Red Channel'])

# Plot the normalized histogram for the B&W image
ax4.set_xlim(200, 256)
ax4.plot(hist_bw_normalized, color='black')
ax4.set_xlabel('Pixel Value')
ax4.set_ylabel('Normalized Frequency')
ax4.set_title('Zoomed Histogram of B&W Image')


plt.tight_layout()
plt.show()

By using the Kmeans algorithm with 6 centroids we can see that, on average, the H&E images have a color distribution higly skewed around purple tones.

In [ ]:
image = cv2.imread("./data/66-4/processed_32/H21-066.4_HE332_001_Scan1.tif")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
pixels = image_rgb.reshape((-1, 3))
num_colors = 6
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
_, labels, centers = cv2.kmeans(pixels.astype(np.float32), num_colors, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)
reduced_image = centers[labels.flatten()]
palette = centers.reshape(1, -1, 3)
palette = cv2.cvtColor(palette, cv2.COLOR_BGR2RGB)
rotated_img = ndimage.rotate(palette, 90)
reduced_image = reduced_image.reshape(image_rgb.shape)
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5))
ax1.imshow(reduced_image)
ax2.imshow(rotated_img, cmap=plt.cm.gray)
plt.title('Color Palette')
plt.axis('off')
plt.tight_layout()
plt.show()


## Image Labels

In this section we will discuss the elements of the H&E images and for that we are going to look into the labels. They are stored in a geojson format, where geopandas can come in handy.
At this point we have 6 labelled images corresponding to slides:
- 033
- 089
- 105
- 137
- 225
- 285

Lets have a look one of them: 

In [ ]:

resolution = "32"
image = load_image('./data/66-4/processed_'+resolution+'/H21-066.4_HE332_033_Scan1.tif')
labels_df  = load_labels('./data/GeoJson/H21-066.4_HE332_033_Scan1.qptiff - resolution #1.geojson', int(resolution))

class_colors = {'epithelium':"#ff0000", 
              'blood_vessels':"#0000ff", 
              'stroma':"#00ff00", 
              'adipocytes':"#9000ff"}

# Plot each geometry with a color depending on the class
legend_handles = []
fig, ax = plt.subplots(figsize=(10, 10))
for class_label, color in class_colors.items():
    subset = labels_df[labels_df['label'] == class_label]
    subset.plot(ax=ax, color=color, label=class_label)
    legend_handles.append(mpatches.Patch(color=color, label=class_label))

plt.legend(handles=legend_handles)
plt.imshow(image)
plt.show()

Lets look at the labels closely and see how they look like. They are also some other elements like air bubbles and dust worth mentioning as they represent a source of noise in the samples:

In [ ]:
# CODE for saving the cropped images 

# resolution = "original"

# image = load_image('./data/66-4/H21-066.4_HE332_033/processed_'+resolution+'/H21-066.4_HE332_033_Scan1.tif')
# labels_df  = load_labels('./data/GeoJson/H21-066.4_HE332_033_Scan1.qptiff - resolution #1.geojson')
# label = "adipocytes"

# polygon_gdf = labels_df[labels_df["label"]==label]["geometry"].iloc[0]
# # Get the bounding box of the polygon
# min_x, min_y, max_x, max_y = np.floor(polygon_gdf.bounds).astype("int64")
# crop = image[min_y:max_y, min_x:max_x]

# translate_x = ((max_x-min_x) - polygon_gdf.bounds[0] - polygon_gdf.bounds[2]) / 2
# translate_y = ((max_y-min_y) - polygon_gdf.bounds[1] - polygon_gdf.bounds[3]) / 2
# translated_polygon = translate(polygon_gdf, xoff=translate_x, yoff=translate_y)

# contour = np.array([[int(x), int(y)] for x, y in translated_polygon.exterior.coords])

# mask = np.zeros_like(crop)*255
# cv2.fillPoly(mask, pts=[contour], color=(255,255,255))
# masked_img = cv2.bitwise_and(crop, mask)

# cv2.imwrite((label+".png"), masked_img)

In [ ]:
fig, ((ax1, ax2, ax5), (ax3, ax4, ax6)) = plt.subplots(2, 3, figsize=(10, 6))
img = load_image("./data/66-4/processed_32/H21-066.4_HE332_001_Scan1.tif")
adipocytes = load_image('./misc/adipocytes.png')
blood_vessels = load_image("./misc/epithelium.png")
stroma = load_image("./misc/stroma.png")
epithelium = load_image("./misc/blood_vessels.png")
ax1.set_title("Adipocytes", y=-0.2)
ax1.imshow(adipocytes)
ax1.axis('off')
ax2.set_title("Blood Vessels", y=-0.2)
ax2.imshow(blood_vessels)
ax2.axis('off')
ax3.set_title("Stroma", y=-0.15)
ax3.imshow(stroma)
ax3.axis('off')
ax4.set_title("Epithelium", y=-0.2)
ax4.imshow(epithelium)
ax4.axis('off')
ax5.imshow(img[1450:1800, 1200:1600])
ax5.set_title("Air Bubble", y=-0.15)
ax5.axis('off')
ax6.imshow(img[550:700, 50:200])
ax6.set_title("Dark Hole", y=-0.15)
ax6.axis('off')
plt.savefig("labels_and_artifacts.png", dpi=600)
plt.show()

We can see that some labels are significantly different in size. This will pose a problem in the future as we will we have to deal with class imbalance and overfitting. As we will work with some downsized resolutions the large labels will look the same while others might lose a lot of information:

Epithelium Downsizing Example

In [ ]:
# Initialize an empty list to store images
images = []

# Loop through files in the directory
for file in os.listdir("./misc"):
    if "smallSeg" in file:
        image = cv2.imread(os.path.join("./misc", file))
        # Resize the image to a common size (e.g., 200x200)
        resized_image = cv2.resize(image, (1000, 1000))  # Adjust dimensions as needed
        # Append the resized image to the list
        images.append(resized_image)

# Concatenate the images horizontally
concatenated_images = cv2.hconcat(images)

plt.figure(figsize=(15, 5)) 
# Show each image with a title
list = ["Original", "x2", "x4", "x8"]

for i, image in enumerate(images):
    plt.subplot(1, len(images), i + 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(list[i], y=-0.12, fontsize=18)  # Add a title to the image
    plt.axis('off')
# Show the concatenated image

plt.show()

Adypocytes downsample example

In [ ]:
# Initialize an empty list to store images
images = []

# Loop through files in the directory
for file in os.listdir():
    if "bigSeg" in file:
        image = cv2.imread(file)
        # Resize the image to a common size (e.g., 200x200)
        resized_image = cv2.resize(image, (1000, 1000))  # Adjust dimensions as needed
        # Append the resized image to the list
        images.append(resized_image)

# Concatenate the images horizontally
concatenated_images = cv2.hconcat(images)

plt.figure(figsize=(15, 5)) 
# Show each image with a title
list = ["Original", "x2", "x4", "x8"]

for i, image in enumerate(images):
    plt.subplot(1, len(images), i + 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(list[i])  # Add a title to the image
    plt.axis('off')
# Show the concatenated image
plt.show()

To wrap up lets look at the pixel distribution per label. For that we will extract all the labels for the 6 images in a dataframe and will count the pixels different than zero, which correspond to background

In [ ]:
result_df = pd.DataFrame(columns=['geometry', 'label', 'encoded_label'])
for f in os.listdir("./data/GeoJson"):
    df = load_labels(os.path.join("./data/GeoJson",f))
    result_df = pd.concat([result_df, df], axis=0)

# Plot the barplot with seaborn
sns.countplot(y=result_df["label"], palette="viridis")

plt.xlabel("Count")
plt.ylabel("Label")
plt.xticks(rotation=45)
plt.title("Distribution of class labels")
plt.show()

In [ ]:
mask_list = []
rgb_list = []
for f in tqdm_notebook(image_files):
    
    image = cv2.imread(f)
    rgb_list.append(image)

    # Convert the color image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold the grayscale image to get a binary mask of non-black pixels
    _, mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)
    # Convert the binary mask to a matrix with labels
    labels_matrix = np.where(mask > 0, get_label_index(f), 0)
    mask_list.append(labels_matrix)

In [ ]:
# Initialize the global image matrix
matrix_size = 1028
global_image = np.zeros((matrix_size, matrix_size))
global_rgb = np.zeros((matrix_size, matrix_size, 3), dtype=np.uint8)


check = 0
x_mark, y_mark = 0, 0

# Iterate over each mask
for i, labels_matrix in enumerate(mask_list):
    #print(i)
    #x_mark, y_mark = 0, 0
    if check == 1:
        break
    
    # Iterate over rows of the global image
    while x_mark + labels_matrix.shape[0] < global_image.shape[0]:
        # Check if the mask fits horizontally without going out of bounds
        if y_mark + labels_matrix.shape[1] > global_image.shape[1]:
            # Move to the next row
            x_mark += 1
            y_mark = 0
            if x_mark == 1028 and y_mark == 1028:
                check = 1
                break
            continue
        
        # Check if the current position in the global image is empty
        if np.all(global_image[x_mark:x_mark+labels_matrix.shape[0], y_mark:y_mark+labels_matrix.shape[1]] == 0):
            # Place the mask into the global image
            global_image[x_mark:x_mark+labels_matrix.shape[0], y_mark:y_mark+labels_matrix.shape[1]] += labels_matrix
            global_rgb[x_mark:x_mark+labels_matrix.shape[0], y_mark:y_mark+labels_matrix.shape[1]] += rgb_list[i]
            break
        
        # Move to the next column
        y_mark += 1
    
unique_values, counts = np.unique(global_image, return_counts=True)

# Create a dictionary to store the counts for each value
count_dict = dict(zip(unique_values, counts))
key_replacements = {0.0:"background", 1.0:'epithelium', 2.0:'blood_vessels', 3.0:'stroma', 4.0:'adipocytes'}
new_dict = {key_replacements.get(k, k): v for k, v in count_dict.items()}

df = pd.DataFrame(list(new_dict.items()), columns=['Label', 'Count'])
total_pixels = sum(df["Count"])
df["percentage"] = np.round(df["Count"]/total_pixels,2)


In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

im1 = axes[0, 0].imshow(global_image, cmap='viridis')
fig.colorbar(im1, ax=axes[0, 0])
axes[0, 0].set_title('Grayscale Image')

axes[0, 1].imshow(global_rgb)
axes[0, 1].set_title('RGB Image')

# Bar plot using Seaborn
sns.barplot(x='Label', y='Count', data=df, ax=axes[1, 0], color='skyblue')
axes[1, 0].set_title('Distribution of pixels per label')
axes[1, 0].set_ylabel('Count')

# Sample data
label = df["Label"].values
colors = ['gray', '#ff0000', '#0000ff', '#00ff00', "#9000ff"]
sizes = df["percentage"].values


axes[1, 1].pie(sizes, labels=label, colors=colors, autopct='%1.1f%%', startangle=90, pctdistance=1.2, textprops={'size': 8},wedgeprops=dict(edgecolor='black'), labeldistance=None, radius=1)
axes[1, 1].set_title('Distribution of pixels per label')
axes[1, 1].legend(labels, loc="upper left", bbox_to_anchor=(-0.5,1))


plt.title('% of pixels per label')

plt.show()

With this EDA we can conclude the following:
- We need to account for outliers while reading the image and crop only the regions of interest.
- The codensed distribution of colors around pinks and blues might make the classification task challenging in terms of differences in pixel intensities.
- The label imbalance needs to be handled with data augmentation or additional labeling effort from the side of the researchers.